                  Detecting Malaria cells using Convolutional Neural Network

# Importing Libraries

In [ ]:
import  os
from keras.preprocessing.image import ImageDataGenerator, array_to_img,img_to_array,load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import cvutils
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import keras
from keras.utils import plot_model
from keras.layers import BatchNormalization
from pathlib import Path
from keras.preprocessing import image
from keras.models import model_from_json

# Loading Image Dataset

In [ ]:
data    = []
labels  = []
Parasitized  = os.listdir("cell_images/Parasitized/")
Uninfected   = os.listdir("cell_images/Uninfected/")

In [ ]:
for i in Parasitized:
    if i != "Thumbs.db":
        image = cv2.imread("cell_images/Parasitized/" + i)
        size_image = cv2.resize(image, (50, 50))
        data.append(np.array(size_image))
        labels.append(1)
for j in Uninfected:
    if j != "Thumbs.db":
        image = cv2.imread("cell_images/Uninfected/" + j)
        size_image = cv2.resize(image, (50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    

In [ ]:
print(len(data))
print(len(labels))

# Sample Data visualization

In [ ]:
cells = np.array(data)
labels = np.array(labels)

np.save('Cells' , cells)
np.save('Labels' , labels)

idx = np.arange(cells.shape[0])
np.random.shuffle(idx)
cells = cells[idx]
labels = labels[idx]

In [ ]:
plt.figure(1 , figsize = (15 , 9))
n = 0 
for i in range(49):
    n += 1 
    r = np.random.randint(0 , cells.shape[0] , 1)
    plt.subplot(7 , 7 , n)
    plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
    plt.imshow(cells[r[0]])
    plt.title('{} : {}'.format('Infected' if labels[r[0]] == 1 else 'UnInfected' ,
                               labels[r[0]]) )
    plt.xticks([]) , plt.yticks([])
    
plt.show()

# Normalize data set

In [ ]:
# split data to 80% train and 20% test
x_train, x_test, y_train, y_test = train_test_split(cells, labels, test_size = 0.2, random_state = 101) 
#split train data into validation data and train (70% train, 10% validation, 20% test)
#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_val   = x_val.astype('float32')
x_train/255
x_test/255
x_val/255

In [ ]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train,2)
y_test  = keras.utils.to_categorical(y_test,2)
y_val  = keras.utils.to_categorical(y_val,2)

# Create a model and add layers

In [ ]:

model = Sequential()

In [ ]:
model.add(Conv2D(32,kernel_size = (3,3),padding = 'same',input_shape = (50,50,3),activation = "relu"))
#model.add(Conv2D(32,kernel_size = (3,3),activation = "relu")) #k
model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(BatchNormalization(axis = -1))
#model.add(Dropout(0.25))  # activated now


model.add(Conv2D(64,kernel_size = (3,3),padding = 'same',activation = "relu"))
#model.add(Conv2D(64,kernel_size = (3,3),activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(BatchNormalization(axis = -1))
#model.add(Dropout(0.25)) 


#model.add(Conv2D(64,kernel_size = (3,3),padding = 'same',activation = "relu"))
#model.add(Conv2D(64,kernel_size = (3,3),activation = "relu"))
#model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(BatchNormalization(axis = -1))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(110, activation = "relu"))
#model.add(Dropout(0.5))
model.add(Dense(2,activation = "softmax"))


# Print a summary of the model
model.summary()


# Compile the model

In [ ]:
model.compile(
                loss= 'categorical_crossentropy',
                optimizer= "adam",
                metrics= ['accuracy']
             )

# Train the model

In [ ]:
hist = model.fit(
            x_train,
            y_train,
            batch_size= 50,
            epochs= 5,
            validation_data= (x_val,y_val),
            shuffle= True
         )

In [ ]:
import pandas as pd
acc = pd.DataFrame.from_dict(hist.history)
acc = pd.concat([pd.Series(range(0,30),name='epochs'),acc],axis=1)
acc.head()

# Plot Model Accuracy

In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Save neural network structure

In [ ]:
model_Struct = model.to_json()
file_name = Path("model_structure.json")
file_name.write_text(model_Struct)

# Save neural network's trained weights

In [ ]:
model.save_weights("model_weights.h5")

# Predictions

In [ ]:
ypredict = model.predict(x_test)

In [ ]:
print(ypredict)

# Validating Model prediction

* Validating the model by passing single image

In [ ]:
class_labels = ["Uninfected","Infected"]
from keras.preprocessing import image
#img = image.load_img("cell_images/Parasitized/C37BP2_thinF_IMG_20150620_131423a_cell_92.png", target_size=(50, 50))
img = image.load_img("cell_images/Uninfected/C1_thinF_IMG_20150604_104722_cell_123.png", target_size=(50, 50))
image_to_test = image.img_to_array(img)
list_of_images = np.expand_dims(image_to_test, axis=0)
results = model.predict(list_of_images)
single_result = results[0]
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]
#print(most_likely_class_index)
class_label = class_labels[most_likely_class_index]
img
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))
plt.imshow(img)
plt.show()

* Validating the model by loading model structure

In [ ]:
class_labels = ["Uninfected","Infected"]

# Load the json file that contains the model's structure
f = Path("model_Structure.json")
model_Structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_Structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img("cell_images/Parasitized/C38P3thinF_original_IMG_20150621_112116_cell_204.png", target_size = (50,50))
#img = image.load_img("cell_images/Uninfected/C1_thinF_IMG_20150604_104722_cell_123.png", target_size = (50,50))
# Convert the image to a numpy array
img_to_test = image.img_to_array(img)

# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(img_to_test,axis= 0)

# Make a prediction using the model
results = model.predict(list_of_images)

# Since we are only testing one image, we only need to check the first result
single_result = results[0]
# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_indx = int(np.argmax(single_result))
class_likhood = single_result[most_likely_class_indx]


# Get the name of the most likely class
class_label = class_labels[most_likely_class_indx]

# Print the result

print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likhood))
plt.imshow(img)
plt.show()

# Making Prediction using Transfer Learning

In [1]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np
from keras.applications import vgg16
from keras import applications

Using TensorFlow backend.


In [2]:
model = applications.VGG16(include_top= False, weights= 'imagenet')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
bloc

In [3]:
file_path = Path("model_structure.json")
model_struct_Pretrain = file_path.read_text()
Model_pretrained = model_from_json(model_struct_Pretrain)
Model_pretrained.load_weights("model_weights.h5",by_name=True)

In [4]:
img = image.load_img("cell_images/Uninfected/C1_thinF_IMG_20150604_104722_cell_123.png")

In [5]:
img_array_pre = image.img_to_array(img)
images = np.expand_dims(img_array_pre, axis = 0)
#images = vgg16.preprocess_input(images)

In [6]:
# Normalize the data
images = vgg16.preprocess_input(images)

# Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(50, 50, 3))
features = feature_extraction_model.predict(images)

# Given the extracted features, make a final prediction using our own model
#results = model.predict(features)

# Since we are only testing one image with possible class, we only need to check the first result's first element
#single_result = results[0][0]

# Print the result
print("Likelihood that this image contains a dog: {}%".format(int(single_result * 100)))

ValueError: Error when checking : expected input_2 to have shape (50, 50, 3) but got array with shape (100, 136, 3)

In [ ]:
pwd

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

In [ ]:
#from imagenet_utils import decode_predictions
print('Predicted:', decode_predictions(block4_pool_features))

In [ ]:
from keras.applications import imagenet_utils
#from imagenet_utils import decode_predictions

In [ ]:
imagenet_utils.decode_predictions(block4_pool_features)

In [ ]:
base_model= vgg16.VGG16(include_top=False, weights=None,input_shape=(150,150,3), pooling='avg')

In [ ]:
base_model.load_weights("model_weights.h5")